<a href="https://colab.research.google.com/github/ICSM/pgmuvi/blob/main/pgmuvi_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use pgmuvi - a brief introduction



`pgmuvi` is a package to infer the multiwavelength behaviour of astronomical lightcurves using Gaussian Processes. It uses the `torch` and `gpytorch` libraries to define GP models, and the probabilistic programming language `pyro` to perform optimisation and sampling of the parameter space. 

## Why pgmuvi

Many packages exist to interpret lightcurves with GPs, so why do we need another one? `pgmuvi` builds on the latest highly-optimised libraries which allow inference with arbitrary GP kernels. This makes it feasible to implement models that are very general, which is important in astronomy where lightcurves can have a wide range of different properties. However, this normally results in a speed penalty, since naive GPs scale with $\mathcal{O}\left(n^3\right)$; to combat this, the libraries implement scalable approaches to GP inference, such as GPU computing. This allows both a wide range of GP kernels and fast inference, meaning that `pgmuvi` can be applied to arbitrary astronomical lightcurves without taking _that much_ longer than codes which are restricted to special cases but blisteringly fast in those cases.

## What does pgmuvi do

The objective is to understand what the PSD of the variability is, and in the case of (quasi-)periodic variables determine their dominant periods. To do this, `pgmuvi` uses a `SpectralMixtureKernel`, which models the PSD of the covariances of the GP as a Gaussian Mixture model. This is a very flexible model which is able to represent a wide range of different kinds of behaviour, so can be useful for a wide range of astronomical lightcurves.

## How to use pgmuvi

The rest of this notebook is basically a quickstart guide to `pgmuvi`. It generates sythetic observations, but you can replace these with reading in your own data and passing it to `pgmuvi`'s `Lightcurve` classes to apply it to your own data.

In [ ]:
try: #This won't work right now - instead clone the repository and `pip install -e .`
    import pgmuvi
except ImportError:
    !pip install pgmuvi
    import pgmuvi

Now we have imported/installed pgmuvi, we can use it to fit some data. For the purposes of this tutorial, we will generate two synthetic datasets - a single-wavelength lightcurve with very simple behaviour, and a second multi-wavelength one with some more complex patterns in it. It is important to note that the data must be in the form of `torch` tensors, rather than numpy arrays or other data types. 

In [ ]:
import torch
import numpy as np
""" Let's generate some synthetic data from a perturbed sine curve 
    but on the same time sampling as the real data"""

P = np.random.uniform(30, 300)#137. #Days!
print("True period: ",P," days")
n_data = 400
jd_min = 2450000
n_periods = np.random.uniform(3,10)
jd_max = jd_min + P*(n_periods)
print("Simulating for ",n_periods," periods")

#train_mag = 
#train_mag = train_mag + 0.1*torch.randn_like(train_mag)
#train_mag_err = 0.1*train_mag

period_guess = P*(np.random.uniform()+0.5)#147 #this number is in the same units as our original input.

#generate data from a simple case - superimpose two sine curves and add noise
timestamps_1d = torch.Tensor(np.random.uniform(jd_min, jd_max, size=n_data))#generate random x data here
fluxes_1d = torch.sin(timestamps_1d*(2*np.pi/P))#generate random y data here
fluxes_1d += 0.1*torch.randn_like(fluxes_1d)
flux_err_1d = 0.1*fluxes_1d


#now generate some data from a more complex case, maybe drawing from a specific PSD to test if pgmuvi reconstructs it
#timestamps_2d = #generate random x data here
#wavelengths_2d = #generate random x data here
#fluxes_2d = #generate random y data here
#flux_err_2d = 

True period:  261.90817683645514  days
Simulating for  3.505045582883577  periods


Now that we've got some simple data, we can feed it to pgmuvi and see how it handles it.

In [ ]:
from pgmuvi.lightcurve import Lightcurve

lightcurve_1d = Lightcurve(timestamps_1d, fluxes_1d, yerr = flux_err_1d, xtransform='minmax')

The `xtransform` argument is important. The computing time and numerical stability of this GP model depends somewhat on the range of frequencies it is asked to compute the PSD on, so it is common to transform the inputs to some simple interval that ensures the frequencies are easier. Astronomical timeseries often cover very long times, and therefore very low frequencies if left in their original units (e.g. Julian Dates), which are particularly difficult to compute on. By default, `Lightcurve` therefore transforms the data so that it is in the closed interval $\left[0, 1\right]$, although other transformations (or none) can be defined by the user.

Now that we have a lightcurve, we can try to fit it:

In [ ]:
lightcurve_1d.fit(model='1D', likelihood='learn')

this is a very simple case, and we have not specified any of the extra keywords that `fit` exposes. You will probably want to specify an initial guess of the GP hyperparameters, and the default number of components in the mixture model for the PSD is 4. Each mixture has a mean, scale and weight for the Gaussian, and there is a mean function (a constant mean) and an extra free parameter to learn additional noise in the input data.

Now that the fit is completed, you can manually inspect the results

In [ ]:
lightcurve_1d.results

but this output is quite ugly. Instead, you can call some routines to make informative summaries of them that are easier to read

In [ ]:
lightcurve_1d.print_results()

and you can also generate some plots of the output; for example the lightcurve with the fit results on top of it

In [ ]:
lightcurve_1d.plot()

or the maximum a-posteriori PSD that pgmuvi has inferred

In [ ]:
lightcurve_1d.plot_psd()

If you want more information, you can use MCMC to sample the posterior of the model (this is still being developed!)

In [ ]:
lightcurve_1d.mcmc()

This takes a while, unless you run it using a GPU - if that works. That's a subject for a different tutorial. Now we've got some results from MCMC, we should take a look at them. First, we can look at the trace

In [ ]:
#internally, Lightcurve should call arviz and produce these outputs
lightcurve_1d.summary()
lightcurve_1d.plot_trace() 
lightcurve_1d.pairplot()

Now if you plot the lightcurve and results, you can get samples drawn from the MCMC chains instead of the mean and credible interval of the MAP solution

In [ ]:
lightcurve_1d.plot(mcmc_samples = True)

and you can do the same thing with the PSD

In [ ]:
lightcurve_1d.plot_psd(mcmc_samples = True)